# Exponential Smoothing: R
First Expermiments

In [ ]:
install.packages("data.table")
install.packages("foreach")
install.packages("doParallel")
install.packages("iterators")
install.packages("bit64")



Die heruntergeladenen Bin"arpakete sind in 
	/var/folders/pm/6f9n9_w51yd38ctgl768_xlh0000gn/T//RtmplcdFgE/downloaded_packages


installiere auch Abh"angigkeit 'iterators'





Die heruntergeladenen Bin"arpakete sind in 
	/var/folders/pm/6f9n9_w51yd38ctgl768_xlh0000gn/T//RtmplcdFgE/downloaded_packages

Die heruntergeladenen Bin"arpakete sind in 
	/var/folders/pm/6f9n9_w51yd38ctgl768_xlh0000gn/T//RtmplcdFgE/downloaded_packages

Die heruntergeladenen Bin"arpakete sind in 
	/var/folders/pm/6f9n9_w51yd38ctgl768_xlh0000gn/T//RtmplcdFgE/downloaded_packages


installiere auch Abh"angigkeit 'bit'





Die heruntergeladenen Bin"arpakete sind in 
	/var/folders/pm/6f9n9_w51yd38ctgl768_xlh0000gn/T//RtmplcdFgE/downloaded_packages


___
___
## Dataset Wind Turbine: Wind Speed

https://www.kaggle.com/datasets/berkerisen/wind-turbine-scada-dataset

In [2]:
# Read the CSV file into a data frame
data <- read.csv("/Users/niklas/Documents/GitHub/Uni/10_Masterarbeit/data_files/wind_turbine_scada.csv", header = TRUE)

# Extract the wind speed column (assuming it's the third column)
wind_speed <- data[, 3]

# Remove NA values
wind_speed <- na.omit(wind_speed)

# Replication factor
factor <- 500

# Replicate the wind_speed vector
replicated_wind_speed <- rep(wind_speed, times = factor)

# Print the size of the replicated data
cat("Dataframe wind_speed has", length(replicated_wind_speed), "rows.")


Dataframe wind_speed has 25265000 rows.

___
### #1 Basic For Loop

In [3]:
# Alpha is the smoothing factor
alpha <- 0.7

# Vector to store execution times
number_of_executions <- 10
execution_times <- numeric(number_of_executions)

# Perform exponential smoothing on the data
for (i in 1:number_of_executions) {
  start_time <- Sys.time()

  # Initialize the smoothed value with the first data point
  smoothed_value <- wind_speed[1]

  # Perform exponential smoothing on the data
  for (j in 2:length(wind_speed)) {
    smoothed_value <- alpha * wind_speed[j] + (1 - alpha) * smoothed_value
  }

  # Stop the timer
  end_time <- Sys.time()
  execution_times[i] <- as.numeric(difftime(end_time, start_time, units = "secs")) # nolint
}

# Calculate the function time
function_time <- median(execution_times)

# Print the results
cat("### Dataset wind_speed\n")
cat("### #1 Basic For Loop \n\n")
cat("The last smoothed value for wind_speed is:", smoothed_value, "\n")
cat(sprintf("The function was executed in %f seconds", function_time))


### Dataset wind_speed
### #1 Basic For Loop 

The last smoothed value for wind_speed is: 9.729056 
The function was executed in 0.001644 seconds

___
### #2 Vectorized

In [4]:
# Alpha is the smoothing factor
alpha <- 0.7

# Vector to store execution times
number_of_executions <- 10
execution_times <- numeric(number_of_executions)

# Perform exponential smoothing on the data
for (i in 1:number_of_executions) {
    start_time <- Sys.time()

    # Perform exponential smoothing on the data
    n <- length(wind_speed)
    weights <- alpha * (1 - alpha)**rev(seq_len(n) - 1)
    smoothed <- rev(cumsum(rev(weights * wind_speed)))
    smoothed_value <- smoothed[1] / sum(weights)

    # Stop the timer
    end_time <- Sys.time()
    execution_times[i] <- as.numeric(difftime(end_time, start_time, units = "secs"))
}

# Calculate the function time
function_time <- median(execution_times)

# Print the results
cat("### Dataset wind_speed\n")
cat("### #2 Vectorized \n\n")
cat("The last smoothed value for wind_speed is:", smoothed_value, "\n")
cat(sprintf("The function was executed in %f seconds", function_time))


### Dataset wind_speed
### #2 Vectorized 

The last smoothed value for wind_speed is: 9.729056 
The function was executed in 0.001210 seconds

___
### #3 Parallelized For Loop

In [27]:
library(parallel)

# Define the smoothing function
smooth_wind_speed <- function(wind_speed, alpha) {
    smoothed_value <- wind_speed[1]
    for (i in 2:length(wind_speed)) {
        smoothed_value <- alpha * wind_speed[i] + (1 - alpha) * smoothed_value
    }
    return(smoothed_value)
}

# Alpha is the smoothing factor
alpha <- 0.7

# Number of executions
number_of_executions <- 10

# Vector to store execution times
execution_times <- numeric()

# Set up parallel computing
cores <- detectCores() - 1
cl <- makeCluster(cores)
clusterExport(cl, varlist = c("wind_speed", "alpha", "smooth_wind_speed"))

# Perform exponential smoothing on the data in parallel
results <- parLapply(cl, 1:number_of_executions, function(i) {
  start_time <- Sys.time()

  # Perform exponential smoothing on the data
  last_smoothed_value <- smooth_wind_speed(wind_speed, alpha)

  # Stop the timer
  end_time <- Sys.time()
  execution_time <- as.numeric(difftime(end_time, start_time, units = "secs"))
  return(list(smoothed_value = last_smoothed_value, execution_time = execution_time))
})

# Stop the cluster
stopCluster(cl)

# Extract execution times and calculate the function time
execution_times <- sapply(results, function(x) x$execution_time)
function_time <- median(execution_times)

# Print the results
cat("### Dataset wind_speed\n")
cat("### #3 Parallelized For Loop \n\n")
cat("The last smoothed value for wind_speed is:", results[[1]]$smoothed_value, "\n")
cat(sprintf("The function was executed in %f seconds", function_time))


### Dataset wind_speed
### #3 Parallelized For Loop 

The last smoothed value for wind_speed is: 9.729056 
The function was executed in 0.028300 seconds

___
### #4 Parallelized Vectorized

In [6]:
library(parallel)

# Define the smoothing function with vectorized operations
smooth_wind_speed <- function(wind_speed, alpha) {
  n <- length(wind_speed)
  weights <- alpha * (1 - alpha)**rev(seq_len(n) - 1)
  smoothed <- rev(cumsum(rev(weights * wind_speed)))
  smoothed_value <- smoothed[1] / sum(weights)
  return(smoothed_value)
}

# Alpha is the smoothing factor
alpha <- 0.7

# Number of executions
number_of_executions <- 10

# Vector to store execution times
execution_times <- numeric()

# Set up parallel computing
no_cores <- detectCores() - 1
cl <- makeCluster(no_cores)
clusterExport(cl, varlist = c("wind_speed", "alpha", "smooth_wind_speed"))

# Perform exponential smoothing on the data in parallel
results <- parLapply(cl, 1:number_of_executions, function(i) {
  start_time <- Sys.time()

  # Perform exponential smoothing on the data
  last_smoothed_value <- smooth_wind_speed(wind_speed, alpha)

  # Stop the timer
  end_time <- Sys.time()
  execution_time <- as.numeric(difftime(end_time, start_time, units = "secs"))
  return(list(smoothed_value = last_smoothed_value, execution_time = execution_time))
})

# Stop the cluster
stopCluster(cl)

# Extract execution times and calculate the function time
execution_times <- sapply(results, function(x) x$execution_time)
function_time <- median(execution_times)

# Print the results
cat("### Dataset wind_speed\n")
cat("### #4 Parallelized Vectorized \n\n")
cat("The last smoothed value for wind_speed is:", results[[1]]$smoothed_value, "\n")
cat(sprintf("The function was executed in %f seconds", function_time))


### Dataset wind_speed
### #4 Parallelized Vectorized 

The last smoothed value for wind_speed is: 9.729056 
The function was executed in 0.002033 seconds

___
___
## Energy Dataset: Solar energy production

In [7]:
# Read the CSV file into a data frame
data <- read.csv("/Users/niklas/Documents/GitHub/Uni/10_Masterarbeit/data_files/energy_dataset.csv", header = TRUE)

# Extract the generation_solar column
generation_solar <- data[, 19]

# Remove NA values
generation_solar <- na.omit(generation_solar)

# Replication factor
factor <- 1400

# Replicate the generation_solar vector
generation_solar <- rep(generation_solar, times = factor)

# Print the size of the replicated data
cat("Dataframe generation_solar has", length(generation_solar), "rows.")


Dataframe generation_solar has 49064400 rows.

___
### #1 Basic For Loop

In [8]:
# Alpha is the smoothing factor
alpha <- 0.7

# Vector to store execution times
number_of_executions <- 10
execution_times <- numeric(number_of_executions)

# Perform exponential smoothing on the data
for (i in 1:number_of_executions) {
  start_time <- Sys.time()

  # Initialize the smoothed value with the first data point
  smoothed_value <- generation_solar[1]

  # Perform exponential smoothing on the data
  for (j in 2:length(generation_solar)) {
    smoothed_value <- alpha * generation_solar[j] + (1 - alpha) * smoothed_value
  }

  # Stop the timer
  end_time <- Sys.time()
  execution_times[i] <- as.numeric(difftime(end_time, start_time, units = "secs")) # nolint
}

# Calculate the function time
function_time <- median(execution_times)

# Print the results
cat("### Dataset generation_solar\n")
cat("### #1 Basic For Loop \n\n")
cat("The last smoothed value for generation_solar is:", smoothed_value, "\n")
cat(sprintf("The function was executed in %f seconds", function_time))


### Dataset generation_solar
### #1 Basic For Loop 

The last smoothed value for generation_solar is: 33.09027 
The function was executed in 1.364029 seconds

___
### #2 Vectorized

In [9]:
# Alpha is the smoothing factor
alpha <- 0.7

# Vector to store execution times
number_of_executions <- 10
execution_times <- numeric(number_of_executions)

# Perform exponential smoothing on the data
for (i in 1:number_of_executions) {
    start_time <- Sys.time()

    # Perform exponential smoothing on the data
    n <- length(generation_solar)
    weights <- alpha * (1 - alpha)**rev(seq_len(n) - 1)
    smoothed <- rev(cumsum(rev(weights * generation_solar)))
    smoothed_value <- smoothed[1] / sum(weights)

    # Stop the timer
    end_time <- Sys.time()
    execution_times[i] <- as.numeric(difftime(end_time, start_time, units = "secs"))
}

# Calculate the function time
function_time <- median(execution_times)

# Print the results
cat("### Dataset generation_solar\n")
cat("### #2 Vectorized \n\n")
cat("The last smoothed value for generation_solar is:", smoothed_value, "\n")
cat(sprintf("The function was executed in %f seconds", function_time))


### Dataset generation_solar
### #2 Vectorized 

The last smoothed value for generation_solar is: 33.09027 
The function was executed in 1.958074 seconds

___
### #3 Parallelized For Loop

In [10]:
library(parallel)

# Define the smoothing function
smooth_generation_solar <- function(generation_solar, alpha) {
  smoothed_value <- generation_solar[1]
  for (i in 2:length(generation_solar)) {
    smoothed_value <- alpha * generation_solar[i] + (1 - alpha) * smoothed_value
  }
  return(smoothed_value)
}

# Alpha is the smoothing factor
alpha <- 0.7

# Number of executions
number_of_executions <- 10

# Vector to store execution times
execution_times <- numeric()

# Set up parallel computing
cores <- detectCores() - 1
cl <- makeCluster(cores)
clusterExport(cl, varlist = c("generation_solar", "alpha", "smooth_generation_solar"))

# Perform exponential smoothing on the data in parallel
results <- parLapply(cl, 1:number_of_executions, function(i) {
  start_time <- Sys.time()

  # Perform exponential smoothing on the data
  last_smoothed_value <- smooth_generation_solar(generation_solar, alpha)

  # Stop the timer
  end_time <- Sys.time()
  execution_time <- as.numeric(difftime(end_time, start_time, units = "secs"))
  return(list(smoothed_value = last_smoothed_value, execution_time = execution_time))
})

# Stop the cluster
stopCluster(cl)

# Extract execution times and calculate the function time
execution_times <- sapply(results, function(x) x$execution_time)
function_time <- median(execution_times)

# Print the results
cat("### Dataset generation_solar\n")
cat("### #3 Parallelized For Loop \n\n")
cat("The last smoothed value for generation_solar is:", results[[1]]$smoothed_value, "\n")
cat(sprintf("The function was executed in %f seconds", function_time))


### Dataset generation_solar
### #3 Parallelized For Loop 

The last smoothed value for generation_solar is: 33.09027 
The function was executed in 2.501423 seconds

___
### #4 Parallelized Vectorized

In [11]:
library(parallel)

# Define the smoothing function
smooth_generation_solar <- function(generation_solar, alpha) {
    n <- length(generation_solar)
    weights <- alpha * (1 - alpha) ** rev(seq_len(n) - 1)
    smoothed <- rev(cumsum(rev(weights * generation_solar)))
    smoothed_value <- smoothed[1] / sum(weights)
    return(smoothed_value)
}

# Alpha is the smoothing factor
alpha <- 0.7

# Vector to store execution times
number_of_executions <- 10
execution_times <- numeric(number_of_executions)

# Set up a cluster of cores
no_cores <- detectCores() - 1  # Leave one core free for system processes
cl <- makeCluster(no_cores)

# Export the necessary variables and functions to the cluster
clusterExport(cl, varlist = c("generation_solar", "alpha", "smooth_generation_solar"))

# Perform exponential smoothing on the data
for (i in 1:number_of_executions) {
    start_time <- Sys.time()

    # Perform exponential smoothing on the data
    last_smoothed_value <- smooth_generation_solar(generation_solar, alpha)

    # Stop the timer
    end_time <- Sys.time()
    execution_times[i] <- as.numeric(difftime(end_time, start_time, units = "secs"))
}

# Calculate the function time
function_time <- median(execution_times)

# Stop the cluster
stopCluster(cl)

# Print the results
cat("### Dataset generation_solar\n")
cat("### #4 Parallelized Vectorized \n\n")
cat("The last smoothed value for generation_solar is:", last_smoothed_value, "\n")
cat(sprintf("The function was executed in %f seconds", function_time))


### Dataset generation_solar
### #4 Parallelized Vectorized 

The last smoothed value for generation_solar is: 33.09027 
The function was executed in 1.734327 seconds

___
___
## Dataset: Heart Rate Seconds

https://www.kaggle.com/datasets/arashnic/fitbit

In [12]:
# Read the CSV file into a data frame
data <- read.csv("/Users/niklas/Documents/GitHub/Uni/10_Masterarbeit/data_files/heartrate_seconds_merged.csv", header = TRUE)

# Select the 'HR' column and remove missing values
heart_rate_data <- data[, 3]

# Replication factor
factor <- 90

# Replicate the heart_rate_data vector
heart_rate_data <- rep(heart_rate_data, times = factor)

# Print the size of the replicated data
cat("Dataframe heart_rate_data has", length(heart_rate_data), "rows.")


Dataframe heart_rate_data has 103921290 rows.

___
### #1 Basic For Loop 

In [13]:
# Alpha is the smoothing factor
alpha <- 0.7

# Vector to store execution times
number_of_executions <- 10
execution_times <- numeric(number_of_executions)

# Perform exponential smoothing on the data
for (i in 1:number_of_executions) {
  start_time <- Sys.time()

  # Initialize the smoothed value with the first data point
  smoothed_value <- heart_rate_data[1]

  # Perform exponential smoothing on the data
  for (j in 2:length(heart_rate_data)) {
    smoothed_value <- alpha * heart_rate_data[j] + (1 - alpha) * smoothed_value
  }

  # Stop the timer
  end_time <- Sys.time()
  execution_times[i] <- as.numeric(difftime(end_time, start_time, units = "secs")) # nolint
}

# Calculate the function time
function_time <- median(execution_times)

# Print the results
cat("### Dataset heart_rate_data\n")
cat("### #1 Basic For Loop \n\n")
cat("The last smoothed value for heart_rate_data is:", smoothed_value, "\n")
cat(sprintf("The function was executed in %f seconds", function_time))


### Dataset heart_rate_data
### #1 Basic For Loop 

The last smoothed value for heart_rate_data is: 98.77753 
The function was executed in 2.870266 seconds

___
### #2 Vectorized

In [14]:
# Alpha is the smoothing factor
alpha <- 0.7

# Vector to store execution times
number_of_executions <- 10
execution_times <- numeric(number_of_executions)

# Perform exponential smoothing on the data
for (i in 1:number_of_executions) {
    start_time <- Sys.time()

    # Perform exponential smoothing on the data
    n <- length(heart_rate_data)
    weights <- alpha * (1 - alpha)**rev(seq_len(n) - 1)
    smoothed <- rev(cumsum(rev(weights * heart_rate_data)))
    smoothed_value <- smoothed[1] / sum(weights)

    # Stop the timer
    end_time <- Sys.time()
    execution_times[i] <- as.numeric(difftime(end_time, start_time, units = "secs"))
}

# Calculate the function time
function_time <- median(execution_times)

# Print the results
cat("### Dataset heart_rate_data\n")
cat("### #2 Vectorized \n\n")
cat("The last smoothed value for heart_rate_data is:", smoothed_value, "\n")
cat(sprintf("The function was executed in %f seconds", function_time))


### Dataset heart_rate_data
### #2 Vectorized 

The last smoothed value for heart_rate_data is: 98.77753 
The function was executed in 3.351181 seconds

___
### #3 Parallelized For Loop

In [15]:
library(parallel)

# Define the smoothing function
smooth_heart_rate_data <- function(heart_rate_data, alpha) {
  smoothed_value <- heart_rate_data[1]
  for (i in 2:length(heart_rate_data)) {
    smoothed_value <- alpha * heart_rate_data[i] + (1 - alpha) * smoothed_value
  }
  return(smoothed_value)
}

# Alpha is the smoothing factor
alpha <- 0.7

# Number of executions
number_of_executions <- 10

# Vector to store execution times
execution_times <- numeric()

# Set up parallel computing
cores <- detectCores() - 1
cl <- makeCluster(cores)
clusterExport(cl, varlist = c("heart_rate_data", "alpha", "smooth_heart_rate_data"))

# Perform exponential smoothing on the data in parallel
results <- parLapply(cl, 1:number_of_executions, function(i) {
  start_time <- Sys.time()

  # Perform exponential smoothing on the data
  last_smoothed_value <- smooth_heart_rate_data(heart_rate_data, alpha)

  # Stop the timer
  end_time <- Sys.time()
  execution_time <- as.numeric(difftime(end_time, start_time, units = "secs"))
  return(list(smoothed_value = last_smoothed_value, execution_time = execution_time))
})

# Stop the cluster
stopCluster(cl)

# Extract execution times and calculate the function time
execution_times <- sapply(results, function(x) x$execution_time)
function_time <- median(execution_times)

# Print the results
cat("### Dataset heart_rate_data\n")
cat("### #3 Parallelized For Loop \n\n")
cat("The last smoothed value for heart_rate_data is:", results[[1]]$smoothed_value, "\n")
cat(sprintf("The function was executed in %f seconds", function_time))


### Dataset heart_rate_data
### #3 Parallelized For Loop 

The last smoothed value for heart_rate_data is: 98.77753 
The function was executed in 5.067736 seconds

___
### #4 Parallelized Vectorized

In [16]:
library(parallel)

# Define the smoothing function
smooth_heart_rate <- function(heart_rate_data, alpha) {
    n <- length(heart_rate_data)
    weights <- alpha * (1 - alpha) ** rev(seq_len(n) - 1)
    smoothed <- rev(cumsum(rev(weights * heart_rate_data)))
    smoothed_value <- smoothed[1] / sum(weights)
    return(smoothed_value)
}

# Alpha is the smoothing factor
alpha <- 0.7

# Vector to store execution times
number_of_executions <- 10
execution_times <- numeric(number_of_executions)

# Set up a cluster of cores
no_cores <- detectCores() - 1  # Leave one core free for system processes
cl <- makeCluster(no_cores)

# Export the necessary variables and functions to the cluster
clusterExport(cl, varlist = c("heart_rate_data", "alpha", "smooth_heart_rate_data"))

# Perform exponential smoothing on the data
for (i in 1:number_of_executions) {
    start_time <- Sys.time()

    # Perform exponential smoothing on the data
    last_smoothed_value <- smooth_heart_rate_data(heart_rate_data, alpha)

    # Stop the timer
    end_time <- Sys.time()
    execution_times[i] <- as.numeric(difftime(end_time, start_time, units = "secs"))
}

# Calculate the function time
function_time <- median(execution_times)

# Stop the cluster
stopCluster(cl)

# Print the results
cat("### Dataset heart_rate_data\n")
cat("### #4 Parallelized Vectorized \n\n")
cat("The last smoothed value for heart_rate_data is:", last_smoothed_value, "\n")
cat(sprintf("The function was executed in %f seconds", function_time))


### Dataset heart_rate_data
### #4 Parallelized Vectorized 

The last smoothed value for heart_rate_data is: 98.77753 
The function was executed in 2.974612 seconds

___
___